In [1]:
import pandas as pd
import numpy as np

IMPORTATION DES BASES DE DONNEES BRUTES

In [ ]:
# Code à éxécuter sans les nrows
link1 = "./BD/name.basics.tsv.gz"
link2 = "./BD/title.akas.tsv.gz"
link3 = "./BD/title.basics.tsv.gz"
link4 = "./BD/title.crew.tsv.gz"
link5 = "./BD/title.episode.tsv.gz"
link6 = "./BD/title.principals.tsv.gz"
link7 = "./BD/title.ratings.tsv.gz"
df_basic_name = pd.read_csv(link1, sep = "\t", compression = "gzip", nrows = 1000)
df_t_akas = pd.read_csv(link2, sep = "\t", compression = "gzip", nrows = 1000)
df_t_basics = pd.read_csv(link3, sep = "\t", compression = "gzip", nrows = 1000)
df_t_crew = pd.read_csv(link4, sep = "\t", compression = "gzip", nrows = 1000)
df_t_episode = pd.read_csv(link5, sep = "\t", compression = "gzip", nrows = 1000)
df_t_principals = pd.read_csv(link6, sep = "\t", compression = "gzip", nrows = 1000)
df_t_ratings = pd.read_csv(link7, sep = "\t", compression = "gzip", nrows = 1000)

NETTOYAGE DE LA TABLE BASIC NAME

In [5]:
# Création d'une copie :
df_basic_name_Clean = df_basic_name.copy()

# filtrage de toutes les valeurs du primaryName non null
df_basic_name_Clean = df_basic_name_Clean[df_basic_name_Clean['primaryName'].isna() == False]

#Transformation des valeurs \N en NAN
df_basic_name_Clean['deathYear'] = df_basic_name_Clean['deathYear'].replace('\\N', np.nan)
df_basic_name_Clean["deathYear"]=df_basic_name_Clean["deathYear"].astype('Int32')
df_basic_name_Clean['birthYear'] = df_basic_name_Clean['birthYear'].replace('\\N', np.nan)
df_basic_name_Clean["birthYear"]=df_basic_name_Clean["birthYear"].astype('Int32')
df_basic_name_Clean['primaryProfession'] = df_basic_name_Clean['primaryProfession'].replace('\\N', np.nan)
df_basic_name_Clean['knownForTitles'] = df_basic_name_Clean['knownForTitles'].replace('\\N', np.nan)

# Filtrage pour ne garder que les acteurs/actrices/réalisateurs:
df_basic_name_Clean = df_basic_name_Clean[(df_basic_name_Clean['primaryProfession'].str.contains('actor'))|(df_basic_name_Clean['primaryProfession'].str.contains('actress'))|(df_basic_name_Clean['primaryProfession'].str.contains('director'))]

#Transformation colonnes "primaryProfession" et "knownForTitles" en liste:
df_basic_name_Clean["primaryProfession"]= df_basic_name_Clean["primaryProfession"].apply(lambda x : x.strip().split(',') if isinstance(x,str) else x)
df_basic_name_Clean["knownForTitles"]= df_basic_name_Clean["knownForTitles"].apply(lambda y : y.strip().split(',') if isinstance(y,str) else y)

#Filtrage pour élilminer ceux qui sont morts il y a trop longtemps:
df_basic_name_Clean = df_basic_name_Clean[df_basic_name_Clean["deathYear"] > 1960]

#Export nouvelle table en csv:
df_basic_name_Clean.to_csv('df_basic_name_clean.csv', index = False)


NETTOYAGE TABLE TITLE AKAS

In [ ]:
# Création d'une copie :
df_t_akas_Clean = df_t_akas.copy()

# suppression des colonnes qui ne nous intéressent pas :
df_t_akas_Clean = df_t_akas_Clean.drop(columns = ['ordering', 'region', 'language', 'types','attributes', 'isOriginalTitle'])

# Aggrégation des différents titres en une liste pour chaque ID:
df_t_akas_Clean = df_t_akas_Clean.groupby('titleId').agg(list).reset_index()

#Export nouvelle table en csv:
df_t_akas_Clean.to_csv("df_t_akas_clean.csv", index = False)

NETTOYAGE TABLE TITLE BASICS

In [ ]:
#Création d'une copie :
df_t_basics_Clean = df_t_basics.copy()

#Nettoyage des données:
df_t_basics_Clean.duplicated().sum() #==> 0 : aucune ligne duppliquée
df_t_basics_Clean[df_t_basics_Clean['genres'].isna()] # => affiche 633 lignes en NAN
df_t_basics_Clean.isna().sum() # ==> affiche 19 ligne en NA dans primaryTitle et originalTitle 

df_t_basics_Clean.dropna(subset=['genres'], inplace=True) # Suppression des lignes en NA de la colonne genre donc 633
df_t_basics_Clean.dropna(subset=['originalTitle'], inplace=True) # suppression des 19 lignes

#Transformation des valeurs \N en NAN
df_t_basics_Clean['startYear'] = df_t_basics_Clean['startYear'].replace('\\N', np.nan)
df_t_basics_Clean['endYear'] = df_t_basics_Clean['endYear'].replace('\\N', np.nan)
df_t_basics_Clean['runtimeMinutes'] = df_t_basics_Clean['runtimeMinutes'].replace('\\N', np.nan)
df_t_basics_Clean['genres'] = df_t_basics_Clean['genres'].replace('\\N', np.nan)

#Listage des genres:
df_t_basics_Clean['genres'] = df_t_basics_Clean['genres'].apply(lambda x: x.strip().split(',') if isinstance(x, str) else [] )

#Export nouvelle table en csv:
df_t_basics_Clean.to_csv("df_t_basics_clean.csv", index = False)

NETTOYAGE TABLE TITLE CREW

In [ ]:
# Création d'une copie :
df_t_crew_Clean = df_t_crew.copy()

# Exploration des erreurs:
valNull = df_t_crew_Clean.isna().sum() #0 valeurs
duplicated = df_t_crew_Clean.duplicated().sum()  # 0valeurs

#Création de listes:
df_t_crew_Clean['directors'] = df_t_crew_Clean['directors'].apply(lambda x: x.strip().split(',') if isinstance(x,str) else x )
df_t_crew_Clean['writers'] = df_t_crew_Clean['writers'].apply(lambda x: x.strip().split(',') if isinstance(x,str) else x)

#Transformation des valeurs \N en NAN
df_t_crew_Clean['directors'] = df_t_crew_Clean['directors'].replace(r'\N', np.nan)
df_t_crew_Clean['writers'] = df_t_crew_Clean['writers'].replace(r'\N', np.nan)

#Export nouvelle table en csv:
df_t_crew_Clean.to_csv("df_t_crew_clean.csv", index = False)


NETTOYAGE TABLE TITLE EPISODE

In [ ]:
# Création d'une copie :
df_t_episode_Clean= df_t_episode.copy()

# Exploration des erreurs:
valNull = df_t_episode_Clean.isna().sum() #0 valeurs
duplicated = df_t_episode_Clean.duplicated().sum()  # 0valeurs mais y a \N à traiter

#Etape 1 : Transformation des valeurs \N en NAN
df_t_episode_Clean['seasonNumber'] = df_t_episode_Clean ['seasonNumber'].replace(r'\N', np.nan)
df_t_episode_Clean['episodeNumber'] = df_t_episode_Clean ['episodeNumber'].replace(r'\N', np.nan)

#création d'une colonne qui servira lors du merge avec title basic pour distinguer les séries.
df_t_episode_Clean['IdSerie'] = True 

# Etape intremédiaire : copy pour la suppression de colonne:
df_t_episode_Clean1 = df_t_episode_Clean.copy()

#Etape : suppression des colonne qu'on va pas utiliser:
df_t_episode_Clean1 = df_t_episode_Clean1.drop(columns = ['seasonNumber', 'episodeNumber'])

#Export nouvelle table en csv:
df_t_episode_Clean1.to_csv('df_t_episode_clean.csv', index = False)

NETTOYAGE TABLE TITLE PRINCIPALS

In [ ]:
# Création d'une copie :
df_t_principals_Clean = df_t_principals.copy()

#Suppression des colonnes : 'ordering', 'job', 'characters'
df_t_principals_Clean = df_t_principals_Clean.drop(columns =['ordering', 'job', 'characters'])

#Suppression des duplicates dûs aux lignes 'characters'
df_t_principals_Clean = df_t_principals_Clean.drop_duplicates()

#Export nouvelle table en csv:
df_t_principals_Clean.to_csv('df_t_principals_clean.csv', index=False)


NETTOYAGE TABLE TITLE RATING

In [ ]:
#Pas de filtre particulier a faire, simple vérification de la présence de valeurs null:

# Création d'une copie :
df_t_ratings_Clean = df_t_ratings.copy()

df_t_ratings_Clean.isna().sum()

#Export nouvelle table en csv:
df_t_ratings_Clean.to_csv('df_t_ratings_clean.csv', index = False)